# Differential Expression for clinical variables related to T2D

In [1]:
import pandas as pd
import numpy as np

## 1. Load data

In [2]:
combined_table = pd.read_csv('results_2023_10_18/combined_data_table.csv', dtype={'public_client_id': str})

/tmp/ipykernel_1103969/3589811407.py:1: DtypeWarning: Columns (992) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_table = pd.read_csv('results_2023_10_18/combined_data_table.csv', dtype={'public_client_id': str})


In [3]:
combined_table.index = pd.MultiIndex.from_frame(combined_table[['public_client_id', 'days_in_program']])

In [4]:
selected_columns_full = np.loadtxt('results_2023_10_18/selected_columns_full.txt', dtype=str, delimiter='\t')
chem_subset_cols = np.loadtxt('results_2023_10_18/chem_subset_cols.txt', dtype=str, delimiter='\t')
selected_chem_bp_cols = np.loadtxt('results_2023_10_18/selected_chem_bp_cols.txt', dtype=str, delimiter='\t')
selected_prot_cols = np.loadtxt('results_2023_10_18/selected_prot_cols.txt', dtype=str, delimiter='\t')
selected_met_cols = np.loadtxt('results_2023_10_18/selected_met_cols.txt', dtype=str, delimiter='\t')

In [5]:
#  try taking only the first patient?
first_items = []
prev_client = None
for c in combined_table.public_client_id:
    if c == prev_client:
        first_items.append(False)
    else:
        prev_client = c
        first_items.append(True)
first_items = np.array(first_items)

combined_table_unique_subjects = combined_table[first_items]

## 2. Do differential expression for proteins and metabolites and clinical data based on delta-HbA1C

In [7]:
import scipy
import numpy as np

def data_test(data1, data2, test='u', use_fdr=True):
    """
    data1 and data2 are dataframes with the same columns
    Assume that they have the same columns
    NaNs are ignored?
    Do we assume that the data is already log-transformed?
    """
    data1_means = data1.mean(0)
    data2_means = data2.mean(0)
    mean_diff = data1_means - data2_means
    if test == 'u':
        t_test_results = scipy.stats.mannwhitneyu(data1, data2, axis=0, alternative='two-sided')
    elif test == 'w':
        t_test_results = scipy.stats.ranksums(data1, data2, axis=0, alternative='two-sided')
    elif test == 't':
        t_test_results = scipy.stats.ttest_ind(data1, data2, axis=0, alternative='two-sided')
    pvals = t_test_results.pvalue
    if use_fdr:
        pvals = scipy.stats.false_discovery_control(pvals, method='by')
    return mean_diff, pvals

In [8]:
import random

def get_control_set(normal_table, test_table):
    """
    Gets similar patients based on demographic characteristics
    """
    control_rows = []
    included_indices = set()
    all_similar_indices = set()
    for i, row in test_table.iterrows():
        age = row.age
        is_m = row.is_m
        min_age = age - 2
        max_age = age + 2
        similar_rows = normal_table[(normal_table.is_m == is_m) & (normal_table.age >= min_age) & (normal_table.age <= max_age)]
        indices = similar_rows.index.to_list()
        indices = [x for x in indices if x not in included_indices]
        try:
            sample = random.sample(indices, 2)
            included_indices.update(sample)
        except(ValueError):
            print('unable to find matching sample', age, is_m)
            min_age = age - 10
            max_age = age + 10
            similar_rows = normal_table[(normal_table.is_m == is_m) & (normal_table.age >= min_age) & (normal_table.age <= max_age)]
            indices = similar_rows.index.to_list()
            indices = [x for x in indices if x not in included_indices]
            if len(indices) == 1:
                sample = indices
            elif len(indices) >= 2:
                sample = random.sample(indices, 2)
            else:
                print('no matching sample found')
                sample = []
            included_indices.update(sample)
    return included_indices

### Iterating through different clinical variables and thresholds to do tests

In [9]:
def get_diffexp(data, variable, high_threshold, low_threshold,
                selected_prots=selected_prot_cols, selected_mets=selected_met_cols,
                selected_clinical=selected_chem_bp_cols):
    "Compute differential expression"
    data_high = data[data[variable] >= high_threshold]
    print('number of data points for {0}:'.format(variable), len(data_high))
    data_low = data[data[variable] < low_threshold]
    control_indices = get_control_set(data_low, data_high)
    data_control = data_low.loc[list(control_indices)]
    mean_diff, pvals = data_test(data_high[selected_prots], data_control[selected_prots], test='u')
    pvals = pd.Series(pvals, index=mean_diff.index)
    print('number of significant proteins for {0}:'.format(variable), (pvals < 0.05).sum())
    mets_mean_diff, mets_pvals = data_test(data_high[selected_mets], data_control[selected_mets], test='u')
    mets_pvals = pd.Series(mets_pvals, index=mets_mean_diff.index)
    print('number of significant metabolites for {0}:'.format(variable), (mets_pvals < 0.05).sum())
    chems_mean_diff, chems_pvals = data_test(data_high[selected_clinical], data_control[selected_clinical], test='u')
    chems_pvals = pd.Series(chems_pvals, index=chems_mean_diff.index)
    print('number of significant clinical tests for {0}:'.format(variable), (chems_pvals < 0.05).sum())
    return mean_diff, pvals, mets_mean_diff, mets_pvals, chems_mean_diff, chems_pvals

In [10]:
def get_diffexp_class(data, variable,
                selected_prots=selected_prot_cols, selected_mets=selected_met_cols,
                selected_clinical=selected_chem_bp_cols):
    "Compute differential expression based on a binarized class variable"
    data_high = data[data[variable]==1]
    print('number of data points for {0}:'.format(variable), len(data_high))
    data_low = data[data[variable]==0]
    control_indices = get_control_set(data_low, data_high)
    data_control = data_low.loc[list(control_indices)]
    mean_diff, pvals = data_test(data_high[selected_prots], data_control[selected_prots], test='u')
    pvals = pd.Series(pvals, index=mean_diff.index)
    print('number of significant proteins for {0}:'.format(variable), (pvals < 0.05).sum())
    mets_mean_diff, mets_pvals = data_test(data_high[selected_mets], data_control[selected_mets], test='u')
    mets_pvals = pd.Series(mets_pvals, index=mets_mean_diff.index)
    print('number of significant metabolites for {0}:'.format(variable), (mets_pvals < 0.05).sum())
    chems_mean_diff, chems_pvals = data_test(data_high[selected_clinical], data_control[selected_clinical], test='u')
    chems_pvals = pd.Series(chems_pvals, index=chems_mean_diff.index)
    print('number of significant clinical tests for {0}:'.format(variable), (chems_pvals < 0.05).sum())
    return mean_diff, pvals, mets_mean_diff, mets_pvals, chems_mean_diff, chems_pvals

In [12]:
variables = ['d_HbA1C_class', 'd_GLUCOSE_class', 'd_INSULIN_class', 'd_HOMA-IR_class', 'd_GFR_class']
    
# current values - TODO: in the previous analyses, these used all of the tables, not just the data subset. That's not done here.
base_variables = ['GLYCOHEMOGLOBIN A1C', 'GLUCOSE']
base_high_thresholds = [6.5, 125]
base_low_thresholds = [6.5, 125]

In [13]:
prots_mean_diff = []
prots_pvals = []
mets_mean_diff = []
mets_pvals = []
chems_mean_diff = []
chems_pvals = []
diffexp_outputs = []
base_diffexp_outputs = []
for var in variables:
    diffexp_output = get_diffexp_class(combined_table_unique_subjects, var)
    diffexp_outputs.append(diffexp_output)
for var, low, high in zip(base_variables, base_low_thresholds, base_high_thresholds):
    diffexp_output = get_diffexp(combined_table_unique_subjects,
                                                    var, high, low)
    base_diffexp_outputs.append(diffexp_output)

number of data points for d_HbA1C_class: 185
unable to find matching sample 18.0 0.0
number of significant proteins for d_HbA1C_class: 19
number of significant metabolites for d_HbA1C_class: 65
number of significant clinical tests for d_HbA1C_class: 22
number of data points for d_GLUCOSE_class: 250
unable to find matching sample 26.0 0.0
unable to find matching sample 74.0 1.0
unable to find matching sample 78.0 0.0
unable to find matching sample 26.0 0.0
unable to find matching sample 29.0 0.0
unable to find matching sample 26.0 0.0
unable to find matching sample 24.0 0.0
number of significant proteins for d_GLUCOSE_class: 0
number of significant metabolites for d_GLUCOSE_class: 0
number of significant clinical tests for d_GLUCOSE_class: 2
number of data points for d_INSULIN_class: 453
unable to find matching sample 26.0 0.0
unable to find matching sample 30.0 0.0
unable to find matching sample 29.0 0.0
unable to find matching sample 33.0 0.0
unable to find matching sample 26.0 0.0
un

### Exporting protein and metabolite tables

In [14]:
prot_names = pd.read_csv('../arivale_data/arivale_prots.tsv', sep='\t')
met_names = pd.read_csv('../arivale_data/arivale_mets.tsv', sep='\t')

In [15]:
met_names.index = met_names.CHEMICAL_ID
prot_names.index = prot_names['index']

In [16]:
top_prots = prot_names.copy()
top_mets = met_names.copy()

In [17]:
prot_new_cols = []
met_new_cols = []
for var, diffexp_output in zip(variables, diffexp_outputs):
    column1_name = 'positive_' + var + '_mean_diff'
    column2_name = 'positive_' + var + '_pvals'
    top_prots[column1_name] = diffexp_output[0]
    top_prots[column2_name] = diffexp_output[1]
    prot_new_cols += [column1_name, column2_name]
    diffexp_output[2].index = diffexp_output[2].index.astype(int)
    diffexp_output[3].index = diffexp_output[3].index.astype(int)
    top_mets[column1_name] = diffexp_output[2]
    top_mets[column2_name] = diffexp_output[3]
    met_new_cols += [column1_name, column2_name]
    
# TODO: add current variables too
for var, diffexp_output in zip(base_variables, base_diffexp_outputs):
    column1_name = 'high_' + var + '_mean_diff'
    column2_name = 'high_' + var + '_pvals'
    top_prots[column1_name] = diffexp_output[0]
    top_prots[column2_name] = diffexp_output[1]
    prot_new_cols += [column1_name, column2_name]
    diffexp_output[2].index = diffexp_output[2].index.astype(int)
    diffexp_output[3].index = diffexp_output[3].index.astype(int)
    top_mets[column1_name] = diffexp_output[2]
    top_mets[column2_name] = diffexp_output[3]
    met_new_cols += [column1_name, column2_name]

In [18]:
top_prots = top_prots[['name', 'uniprot', 'gene_name'] + prot_new_cols]

In [19]:
top_prots = top_prots[~top_prots.positive_d_HbA1C_class_mean_diff.isna()]

In [21]:
top_prots.to_csv('results_2023_10_18/prots_diffexp_d_hba1c.csv', index=None)

In [22]:
top_mets = top_mets[['index', 'CHEMBL', 'KEGG', 'PUBCHEM'] + met_new_cols]

In [23]:
top_mets = top_mets[~top_mets['positive_d_HbA1C_class_mean_diff'].isna()]

In [25]:
top_mets.to_csv('results_2023_10_18/mets_diffexp_d_hba1c.csv', index=None)

### Export clinical table

In [26]:
clinical_columns = {}
for var, diffexp_output in zip(variables, diffexp_outputs):
    column1_name = 'positive_' + var + '_mean_diff'
    column2_name = 'positive_' + var + '_pvals'
    clinical_columns[column1_name] = diffexp_output[4]
    clinical_columns[column2_name] = diffexp_output[5]
    
for var, diffexp_output in zip(base_variables, base_diffexp_outputs):
    column1_name = 'high_' + var + '_mean_diff'
    column2_name = 'high_' + var + '_pvals'
    clinical_columns[column1_name] = diffexp_output[4]
    clinical_columns[column2_name] = diffexp_output[5]

In [27]:
clinical_diffexp = pd.DataFrame(clinical_columns)

In [28]:
clinical_diffexp.sort_values('positive_d_HbA1C_class_pvals')

,positive_d_HbA1C_class_mean_diff,positive_d_HbA1C_class_pvals,positive_d_GLUCOSE_class_mean_diff,positive_d_GLUCOSE_class_pvals,positive_d_INSULIN_class_mean_diff,positive_d_INSULIN_class_pvals,positive_d_HOMA-IR_class_mean_diff,positive_d_HOMA-IR_class_pvals,positive_d_GFR_class_mean_diff,positive_d_GFR_class_pvals,high_GLYCOHEMOGLOBIN A1C_mean_diff,high_GLYCOHEMOGLOBIN A1C_pvals,high_GLUCOSE_mean_diff,high_GLUCOSE_pvals
MCHC,-0.978108,1.595983e-26,0.14820,1.000000,0.347070,3.467682e-06,0.337403,7.528307e-06,0.267092,0.060265,0.034375,1.000000e+00,0.475000,8.812552e-01
TNF-ALPHA,0.430514,2.601741e-24,-0.43852,1.000000,0.041303,7.898377e-11,0.059137,1.015069e-08,0.369163,0.077810,-2.824063,1.000000e+00,-0.479423,4.889036e-01
GLYCOHEMOGLOBIN A1C,-0.290811,6.024600e-16,0.04700,1.000000,-0.007241,1.000000e+00,-0.012788,1.000000e+00,0.071093,0.489075,1.696875,5.147161e-13,1.515385,4.614941e-08
BASOPHILS ABSOLUTE,0.006541,9.079976e-16,-0.00208,0.685669,-0.001796,2.786807e-05,-0.002438,3.322422e-05,-0.001266,0.812049,-0.002656,1.000000e+00,-0.012308,1.998145e-01
ZINC,-175.602703,2.530183e-15,61.81600,0.084763,82.138532,3.853726e-07,83.261359,1.610602e-07,13.344308,1.000000,75.328125,1.000000e+00,134.480769,8.156304e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"CHOLESTEROL, TOTAL",1.732432,1.000000e+00,0.86600,1.000000,-3.518930,1.000000e+00,-3.840081,1.000000e+00,-4.435236,1.000000,-17.906250,2.213862e-01,-9.942308,1.000000e+00
CARBON DIOXIDE (CO2),0.267568,1.000000e+00,0.15400,1.000000,-0.114145,1.000000e+00,-0.063785,1.000000e+00,-0.075724,1.000000,-0.609375,1.000000e+00,-1.019231,3.767755e-01
BUN/CREAT RATIO,0.118919,1.000000e+00,-0.53000,1.000000,-0.435110,3.439630e-01,-0.346829,6.908400e-01,0.451460,1.000000,0.328125,1.000000e+00,0.615385,1.000000e+00
DHA,0.044865,1.000000e+00,-0.01540,1.000000,0.050085,1.000000e+00,0.037747,1.000000e+00,-0.146511,0.446854,-0.026563,1.000000e+00,-0.078846,1.000000e+00


In [29]:
clinical_diffexp.to_csv('results_2023_10_18/clinical_diffexp_d_hba1c.csv')